In [1]:
import pandas as pd
import numpy as np
import datetime
import imp
import scripts as scr
import os
import re
import multiprocessing
import numpy as np
import pandas as pd
from pytictoc import TicToc
from IPython.display import display
import itertools
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pytictoc'

In [297]:
import numpy as np
import pandas as pd
# import datetime


def transform_cols (df, dict_col_types = None):
    # Расширяйте для необходимых столбцов и их явной типизации
    if dict_col_types is None:
        dict_col_types = {
        'amount_original':(float, 0.0),
        'channel_indicator_desc':(str, u'null'),
        'event_description':(str, u'null'),
        'short_date':(int, 0),
        'cdf_s_20':(str, u'null'),
        'cdf_s_126':(str, u'null'),
        'cdf_s_127':(int, 30),
        'cdf_s_129':(int, 30),
        'cdf_s_138':(str, u'null'),
        'cdf_s_130':(int, 30),
        'cdf_s_133':(int, 30),
        'cdf_s_134':(int, 30),
        'cdf_s_135':(int, 30),
        'cdf_s_140':(float, 0.0),
        'cdf_s_218':(str, u'null'),
        'cdf_s_294':(int, 0),
        'cdf_s_299':(str, u'null'),
        'data_s_65':(int, 0),
        'data_i_120':(int, 0),
        'data_i_154':(float, -150)
        }
                
    if df.shape[0] == 0:
        return df
    
    df.replace(u'null', np.nan, inplace=True)

    for i in dict_col_types:
        if i in df.columns:
            change_type, fill_value = dict_col_types[i]
            df[i] = df[i].fillna(fill_value).astype(change_type)
    
    return df

# def calc_base_features(data):
#     feat_matrix = pd.DataFrame()
#     #data = data[data.event_description.isin([u'Перевод частному лицу',u'Оплата услуг',u'Перевод между своими счетами и картами'])]
    
#     if data.shape[0] == 0:
#         return feat_matrix
    
#     # заполняем ряд пропусков

#     # кумулятивная сумма опреаций за сутки, если не заполнена, то значит это первая операций, т.е. = 0
#     data.cdf_s_140 = data.cdf_s_140.fillna(0).astype(float)/1000
#     data.data_i_120.fillna(1, inplace=True)

    
#     feat_matrix['event_id'] = data.event_id
#     feat_matrix['user_id'] = data.user_id
#     feat_matrix['custom_mark'] = data.custom_mark    
#     feat_matrix['event_time'] = data.event_time
#     feat_matrix['amount'] = data.amount_original

#     feat_matrix['data_i_120'] = data.data_i_120
#     cdf_keep_cols = ['cdf_s_127', 'cdf_s_129', 'cdf_s_130', 'cdf_s_133', 'cdf_s_134', 'cdf_s_135']
#     feat_matrix.loc[:, cdf_keep_cols] = data[cdf_keep_cols]
#     feat_matrix['data_i_120'] = data.data_i_120

#     # кумулятивная сумма операций за сутки в каналах web и МП
#     feat_matrix['cumulative_sum_total'] = data.cdf_s_140

    
#     feat_matrix['client_age'] = [x.days/360 for x in (data.event_time - data.cdf_s_19)]
  
        
#     feat_matrix['cat_new_ip'] = [1 if x == u'ДА' else 0 if x == u'НЕТ' else 2 for x in data.cdf_s_126]
#     feat_matrix['cat_new_prov'] =  [1 if x == u'ДА' else 0 if x == u'НЕТ' else 2 for x in data.cdf_s_138]
#     feat_matrix['channel_op'] =  [0 if x == u'MOBILE' else 1 if x == u'WEB' else 2 for x in data.channel_indicator_desc]
#     feat_matrix['op_type'] = [0 if x == u'Перевод частному лицу' else 1 if x==u'Оплата услуг' else 2 if x ==u'Перевод между своими счетами и картами' else 3 for x in data.event_description]


#     # бинарный флаг определяющий наличие возраста получателя
#     # (полезен для линейных моделей,  менее для деревьев с учетом следующего признака)
#     feat_matrix['recip_age'] = [1 if x == 0 else 0 for x in data.cdf_s_294]
#     # разница возорастов получателей и отправителей, если отсутствует/неприменимо, то padding 500
#     feat_matrix['age_diff'] = feat_matrix.client_age - [int(x) if x != 0 else 1000 for x in data.cdf_s_294]
    
#     feat_matrix['relative'] = [1 if x == u'ДА' else 0 for x in data.cdf_s_218] # перевод родственнику
    
#     feat_matrix['know_recip_power'] = [ x if x is not None else 0 for x in data.data_s_65] # сила связи отправителя и получателя
    

#     feat_matrix['data_i_154'] = [ x if x is not None else -150 for x in data.data_i_154]
#     feat_matrix['know_recip_card_age'] = [1 if x is not None else 0 for x in data.cdf_s_124]
    
    
#     feat_matrix['recip_card_age'] = [x.days if type(x) is not pd.tslib.NaTType else 912321 for x in (data.event_time - data.cdf_s_124)]
    
#     # feat_matrix['cat_client_region'] = [x if x.isdigit() else 912321 for x in data.cdf_s_20]
#     feat_matrix['one_region'] = (data.cdf_s_20 == data.cdf_s_299).astype(int) # сравнение регионов
    

#     #ADD NEW FEATURES
#     feat_matrix['krp_pow2'] = (feat_matrix['know_recip_power']) ** 2
#     feat_matrix['log_amount'] = np.log(feat_matrix['amount'] + 1)
#     feat_matrix['ip_isp'] = np.array([x if x.isdigit() else 912321 for x in data.cdf_s_20], dtype=float)
#     feat_matrix['amnt2chnls'] = (data["amount_original"].fillna(0).astype(float) / \
#         (data["cdf_s_136"].fillna(0).astype(float) + data["amount_original"].fillna(0).astype(float) + \
#             data["amount_original"].fillna(0) + 1))
    
#     # поставил order_cols временно, чтобы сохранить такой же порядок как в оригинальном
#     order_cols = ['event_id', 'user_id', 'custom_mark', 'event_time', 'amount',
#                   'client_age', 'cat_new_ip', 'cat_new_prov', 'channel_op', 'op_type',
#                   'recip_age', 'age_diff', 'cumulative_sum_total', 'data_i_120',
#                   'relative', 'know_recip_power', 'cdf_s_127', 'cdf_s_135', 'cdf_s_130',
#                   'cdf_s_129', 'cdf_s_134', 'data_i_154', 'cdf_s_133',
#                   'know_recip_card_age', 'recip_card_age', 'one_region', 'krp_pow2',
#                   'log_amount', 'ip_isp', 'amnt2chnls']
#     feat_mat = feat_mat[order_cols].copy()
#     return feat_matrix


def load_data(chunk_fnames, fields=None, query=None, sample='train', dict_col_types=None):
    df = pd.DataFrame({})
    if isinstance(chunk_fnames, str):
        chunk_fnames = [chunk_fnames]
        
    for filename in chunk_fnames:
        chunk_df = pd.read_feather(filename)
            
        if fields is None:
            fields = chunk_df.columns.tolist()
        
        transormed = transform_cols(chunk_df)
        
        if query:
            transormed = transormed.query(query)
 
        df = pd.concat([df, transormed[fields]], ignore_index=True)
    return df


def features_handler(chunk_names, calc_feat, query=None, chunk_size=5000):
    res_df = pd.DataFrame()
    
    for chunk_name in chunk_names:
        loaded_data = load_data(chunk_name, query=query, dict_col_types=None)
        feat_chunk = calc_feat(loaded_data)
        res_df = pd.concat([res_df, feat_chunk], ignore_index=True)

    return res_df

def cust_mark_to_class(custom_mark):
    """
    Преобразует входящее значение CUSTOM_MARK в класс
    return:
        1 - фрод
        0 - легитимная
        -1 - неизвестно
    """
    ret = -1
    if custom_mark in ['F','S']:
        ret = 1
    elif custom_mark in ['A','G', np.NaN]:
        ret = 0
    
    return ret

In [292]:
test_folder = '../../data/raw_splits/test/'
test_files = sorted([x for x in os.listdir(test_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
test_files = [os.path.join(test_folder, x) for x in test_files]
print(f'Length of test files is {len(test_files)}')
test_files[:5]

Length of test files is 26


['../../data/raw_splits/test/chunk_0.fth',
 '../../data/raw_splits/test/chunk_1.fth',
 '../../data/raw_splits/test/chunk_2.fth',
 '../../data/raw_splits/test/chunk_3.fth',
 '../../data/raw_splits/test/chunk_4.fth']

In [215]:
FIRST_N = 100
N_THREADS = 5
train_folder = '../../data/raw_splits/train/'
train_files = sorted([x for x in os.listdir(train_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
train_files = [os.path.join(train_folder, x) for x in train_files]
print(f'Length of train files is {len(train_files)}')
train_files[:5]

Length of train files is 51


['../../data/raw_splits/train/chunk_0.fth',
 '../../data/raw_splits/train/chunk_1.fth',
 '../../data/raw_splits/train/chunk_2.fth',
 '../../data/raw_splits/train/chunk_3.fth',
 '../../data/raw_splits/train/chunk_4.fth']

In [47]:
%%time
data_raw = pd.read_feather(train_files[0])

CPU times: user 2.72 s, sys: 1.48 s, total: 4.2 s
Wall time: 4.2 s


---

In [48]:
%%time
data = transform_cols(data_raw)

CPU times: user 32.9 s, sys: 1.49 s, total: 34.4 s
Wall time: 34.4 s


### Подробнее что есть что

'event_id'- уникальный id транзакции

'short_date' - дата (для удобства поиска/агрегации)

'user_id' - уникальный идентификатор клиента

'event_time' - дата и время собятия

'custom_mark'- результат разбора события

'channel_indicator_desc' - канал проведения операции (web, мобильное приложение, SMS-банк и пр.)

'event_description' - описание непосредственно события (например, вход в систему или перевод, оплата услуг)

'amount_original' - сумма в рублях

'user_agent_string_hash', 'browser_plugins_hash', 'screen_hash' - различные признаки устройства (с определенной степенью точности позволяет понять уникальность устройства в разрезе пользователя)

ip_address', 'ip_country', 'ip_region', 'ip_city', 'ip_isp' - данные, связанные с IP (регион, город и интернет-провадйер по БД гео-IP)
'hardwareid' - уникальный идентификатор устройства для канала мобильных приложений

'user_acct_number_hashed'- счет отправителя (хэшированный)
'ext_acct_number_hashed'- счет получателя (хэшированный)


'data_s_65' - результат определения связи между отправителем и получателем (чем больше, тем сильнее связь) 
'data_i_118', 'data_i_119’, 'data_i_120', 'data_i_154' - ряд признаков, которые описывают устройство, с которого проводятся операции
 


 'cdf_s_136','cdf_s_137','cdf_s_140'- кумулятивные суммы операций за сутки в web, МП, web + МП
 'cdf_s_218'- предполагаемое наличие родственной связи 
 'cdf_s_127', 'cdf_s_135', 'cdf_s_130', 'cdf_s_129', 'cdf_s_134', 'cdf_s_128', 'cdf_s_138', 'cdf_s_126' - дней с момента различных рисковых событий



 'cdf_s_19'- ДР клиента
 'cdf_s_20'- Территориальный банк клиента (региональный признак)
 'cdf_s_299'- Тер банк получателя (региональный признак)

 'cdf_s_294'- возраст получателя

 'cdf_s_123'- региональный признак получателя (более локальный по сравнению с ТБ)<br>
 'cdf_s_124'- дата выдачи карты получателя

 'cdf_s_178_hashed' - реквизит получателя

In [50]:
#%%time
tt = TicToc()
tt.tic()

feat_matrix = pd.DataFrame()

if data.shape[0] == 0:
    raise 'shape is 0'

# заполняем ряд пропусков
data.cdf_s_140 = data.cdf_s_140.fillna(0).astype(float) / 1000
data.data_i_120.fillna(1, inplace=True)

feat_matrix['amount'] = data['amount_original']
same_columns = ['event_id', 'user_id', 'event_time', 
                'cdf_s_127', 'cdf_s_129', 'cdf_s_130', 'cdf_s_133', 'cdf_s_134', 'cdf_s_135', 'data_i_120']
for column in same_columns:
    feat_matrix.loc[:, column] = data[column]
feat_matrix['is_fraud'] = [cust_mark_to_class(x) for x in data['custom_mark']]
    

# ----------
# дополнительные фичи
# по user_id
user_id_what_suffix = pd.Series([re.sub('[0-9]', '', x) for x in data['user_id']])
for suffix in ['MBK', 'VSP', 'CRM', 'IVR', 'other']:
    if suffix == 'other':
        result = ~user_id_what_suffix.isin(['MBK', 'VSP', 'CRM', 'IVR', ''])
    else:
        result = (user_id_what_suffix == suffix).astype(int)
    feat_matrix[f'user_id_{suffix}'] = result
feat_matrix['user_id_digit_only'] = feat_matrix['user_id'].apply(lambda x: x.isdigit())
# по каналу ohe
for suffix in ['MOBILEAPI', 'WEBAPI', 'ATMAPI', 'MBK', 'other']:
    if suffix == 'other':
        result = ~data.channel_indicator_desc.isin(['MOBILEAPI', 'WEBAPI', 'ATMAPI', 'MBK'])
    else:
        result = (data.channel_indicator_desc == suffix).astype(int)
    feat_matrix[f'channel_indicator_desc_is_{suffix}'] = result
# время операции
feat_matrix['event_hour'] = [x.hour for x in feat_matrix['event_time']]
feat_matrix['event_hour_night'] = [1 if ((hour >= 23) or (hour <= 7)) else 0 for hour in feat_matrix['event_hour']]
feat_matrix['event_hour_workhour'] = [1 if ((hour >= 8) or (hour <= 17)) else 0 for hour in feat_matrix['event_hour']]
feat_matrix['event_hour_evening'] = [1 if ((hour >= 18) or (hour <= 22)) else 0 for hour in feat_matrix['event_hour']]
    
feat_matrix['event_day'] = [x.dayofweek for x in feat_matrix['event_time']]
feat_matrix['event_day_is_weekend'] = [1 if day >= 6 else 0 for day in feat_matrix['event_day']]
# ----------

tt.toc('First')

First 18.730778 seconds.


In [51]:
# кумулятивная сумма операций за сутки в каналах web и МП, умножил на 1e15 из-за того, что там сильно маленькие числа
feat_matrix['cumulative_sum_total'] = data.cdf_s_140 * 1e15


feat_matrix['client_age'] = [x.days / 365.25 for x in (data.event_time - data.cdf_s_19)]
feat_matrix['client_age_isnull'] = feat_matrix['client_age'].isnull().astype(int)

#заменил на -1
feat_matrix['cat_new_ip'] = [1 if x == 'ДА' else 0 if x == 'НЕТ' else -1 for x in data.cdf_s_126]
feat_matrix['cat_new_prov'] =  [1 if x == 'ДА' else 0 if x == 'НЕТ' else -1 for x in data.cdf_s_138]
feat_matrix['channel_op'] =  [0 if x == 'MOBILE' else 1 if x == 'WEB' else -1 for x in data.channel_indicator_desc]
feat_matrix['op_type'] = [0 if x == 'Перевод частному лицу' else
                          1 if x == 'Оплата услуг' else
                          2 if x == 'Перевод между своими счетами и картами' else
                          3 for x in data.event_description]

tt.toc('Second')

Second 36.774528 seconds.


In [52]:
# бинарный флаг определяющий наличие возраста получателя
# (полезен для линейных моделей,  менее для деревьев с учетом следующего признака)
feat_matrix['transfer_recip_age'] = [1 if x == 0 else 0 for x in data.cdf_s_294]
# разница возрастов получателей и отправителей, если отсутствует/неприменимо, то padding 500
feat_matrix['transfer_age_diff'] = feat_matrix.client_age - [int(x) if x != 0 else 1000 for x in data.cdf_s_294]
# перевод родственнику
feat_matrix['transfer_for_relative'] = [1 if x == 'ДА' else 0 for x in data.cdf_s_218] 
# сила связи отправителя и получателя
feat_matrix['transfer_know_recip_squared'] = [ x if x is not None else 0 for x in data.data_s_65]
# 'data_i_154' - ряд признаков, которые описывают устройство, с которого проводятся операции
feat_matrix['data_i_154'] = [ x if x is not None else -150 for x in data.data_i_154]
# 'cdf_s_124'- дата выдачи карты получателя
feat_matrix['know_recip_card_age'] = ~data.cdf_s_124.isnull().astype(int)
# в cdf_s_124 подмешана дата рождения, поэтому и max
feat_matrix['recip_card_age'] = [max(x.days, 1000) if type(x) is not pd.tslib.NaTType else 1000 
                                 for x in (data.event_time - data.cdf_s_124)]

# feat_matrix['cat_client_region'] = [x if x.isdigit() else 912321 for x in data.cdf_s_20]
feat_matrix['one_region'] = (data.cdf_s_20 == data.cdf_s_299).astype(int) # сравнение регионов

# там с провайдером какая-то фигня до этого была (использовалась левая переменная)
feat_matrix['ip_isp'] = data['ip_isp'].fillna(-1000000).astype(int)
tt.toc('Third')

Third 51.319204 seconds.


In [53]:
# оставшиеся признаки из исходного набора
feat_matrix['know_recip_power'] = [ x if x is not None else 0 for x in data.data_s_65] # сила связи отправителя и получателя
feat_matrix['relative'] = [1 if x == u'ДА' else 0 for x in data.cdf_s_218] # перевод родственнику
feat_matrix['age_diff'] = feat_matrix.client_age - [int(x) if x != 0 else 1000 for x in data.cdf_s_294] # разница возорастов получателей и отправителей, если отсутствует/неприменимо, то padding 500

In [54]:
#ADD NEW FEATURES
feat_matrix['krp_pow2'] = (feat_matrix['know_recip_power']) ** 2
feat_matrix['log_amount'] = np.log(feat_matrix['amount'] + 1)

feat_matrix['client_region_len'] = data.cdf_s_20.apply(lambda x: len(str(x)))
feat_matrix['client_region'] = np.array([x if x.isdigit() else 999999 for x in data.cdf_s_20], dtype=float)
# там какая-то фигня была, переписал по смыслу
# 'cdf_s_136','cdf_s_137','cdf_s_140'- кумулятивные суммы операций за сутки в web, МП, web + МП 
feat_matrix['amnt2chnls'] = (data["amount_original"].fillna(0) / \
    (data["cdf_s_136"].fillna(0).astype(float) + \
     data["cdf_s_136"].fillna(0).astype(float) + \
     data["cdf_s_140"].fillna(0).astype(float) + 1))
tt.toc('Fourth')

Fourth 56.344418 seconds.


In [56]:
# поставил order_cols временно, чтобы сохранить такой же порядок как в оригинальном
order_cols = ['event_id', 'user_id', 'is_fraud', 'event_time', 'amount',
              'client_age', 'cat_new_ip', 'cat_new_prov', 'channel_op', 'op_type', 
              'age_diff', 'cumulative_sum_total', 'data_i_120',
              'relative', 'know_recip_power', 'cdf_s_127', 'cdf_s_135', 'cdf_s_130',
              'cdf_s_129', 'cdf_s_134', 'data_i_154', 'cdf_s_133',
              'know_recip_card_age', 'recip_card_age', 'one_region', 'krp_pow2',
              'log_amount', 'ip_isp', 'amnt2chnls']
feat_matrix = feat_matrix[order_cols].copy()

---

In [61]:
include_channels = ['Перевод частному лицу', 'Оплата услуг', 'Перевод между своими счетами и картами']

In [62]:
include_channels

['Перевод частному лицу',
 'Оплата услуг',
 'Перевод между своими счетами и картами']

In [302]:
def calc_base_features(data, is_train=False):
    feat_matrix = pd.DataFrame()
    
    if data.shape[0] == 0:
        return feat_matrix
    

    feat_matrix = pd.DataFrame()

    if data.shape[0] == 0:
        raise 'shape is 0'

    # заполняем ряд пропусков
    data.cdf_s_140 = data.cdf_s_140.fillna(0).astype(float) / 1000
    data.data_i_120.fillna(1, inplace=True)

    # column for check filtering
    feat_matrix['event_description'] = data['event_description']
    
    feat_matrix['amount'] = data['amount_original']
    same_columns = ['event_id', 'user_id', 'event_time', 
                    'cdf_s_127', 'cdf_s_129', 'cdf_s_130', 'cdf_s_133', 'cdf_s_134', 'cdf_s_135', 'data_i_120']
    for column in same_columns:
        feat_matrix.loc[:, column] = data[column]
    
    # create label only for train dataset
    # feat_matrix['is_fraud'] = [cust_mark_to_class(x) for x in data['custom_mark']]
        
    # ----------
    # дополнительные фичи
    # по user_id
    user_id_what_suffix = pd.Series([re.sub('[0-9]', '', x) for x in data['user_id']])
    for suffix in ['MBK', 'VSP', 'CRM', 'IVR', 'other']:
        if suffix == 'other':
            result = ~user_id_what_suffix.isin(['MBK', 'VSP', 'CRM', 'IVR', ''])
        else:
            result = (user_id_what_suffix == suffix).astype(int)
        feat_matrix[f'user_id_{suffix}'] = result
    feat_matrix['user_id_digit_only'] = feat_matrix['user_id'].apply(lambda x: x.isdigit())
    # по каналу ohe
    for suffix in ['MOBILEAPI', 'WEBAPI', 'ATMAPI', 'MBK', 'other']:
        if suffix == 'other':
            result = ~data.channel_indicator_desc.isin(['MOBILEAPI', 'WEBAPI', 'ATMAPI', 'MBK'])
        else:
            result = (data.channel_indicator_desc == suffix).astype(int)
        feat_matrix[f'channel_indicator_desc_is_{suffix}'] = result
    # время операции
    feat_matrix['event_hour'] = [x.hour for x in feat_matrix['event_time']]
    feat_matrix['event_hour_night'] = [1 if ((hour >= 23) or (hour <= 7)) else 0 for hour in feat_matrix['event_hour']]
    feat_matrix['event_hour_workhour'] = [1 if ((hour >= 8) or (hour <= 17)) else 0 for hour in feat_matrix['event_hour']]
    feat_matrix['event_hour_evening'] = [1 if ((hour >= 18) or (hour <= 22)) else 0 for hour in feat_matrix['event_hour']]
        
    feat_matrix['event_day'] = [x.dayofweek for x in feat_matrix['event_time']]
    feat_matrix['event_day_is_weekend'] = [1 if day >= 6 else 0 for day in feat_matrix['event_day']]
    # ----------
    # кумулятивная сумма операций за сутки в каналах web и МП, умножил на 1e15 из-за того, что там сильно маленькие числа
    feat_matrix['cumulative_sum_total'] = data.cdf_s_140 * 1e15


    feat_matrix['client_age'] = [x.days / 365.25 for x in (data.event_time - data.cdf_s_19)]
    feat_matrix['client_age_isnull'] = feat_matrix['client_age'].isnull().astype(int)

    #заменил на -1
    feat_matrix['cat_new_ip'] = [1 if x == 'ДА' else 0 if x == 'НЕТ' else -1 for x in data.cdf_s_126]
    feat_matrix['cat_new_prov'] =  [1 if x == 'ДА' else 0 if x == 'НЕТ' else -1 for x in data.cdf_s_138]
    feat_matrix['channel_op'] =  [0 if x == 'MOBILE' else 1 if x == 'WEB' else -1 for x in data.channel_indicator_desc]
    feat_matrix['op_type'] = [0 if x == 'Перевод частному лицу' else
                              1 if x == 'Оплата услуг' else
                              2 if x == 'Перевод между своими счетами и картами' else
                              3 for x in data.event_description]

    # бинарный флаг определяющий наличие возраста получателя
    # (полезен для линейных моделей,  менее для деревьев с учетом следующего признака)
    feat_matrix['transfer_recip_age'] = [1 if x == 0 else 0 for x in data.cdf_s_294]
    # разница возрастов получателей и отправителей, если отсутствует/неприменимо, то padding 500
    feat_matrix['transfer_age_diff'] = feat_matrix.client_age - [int(x) if x != 0 else 1000 for x in data.cdf_s_294]
    # перевод родственнику
    feat_matrix['transfer_for_relative'] = [1 if x == 'ДА' else 0 for x in data.cdf_s_218] 
    # сила связи отправителя и получателя
    feat_matrix['transfer_know_recip_squared'] = [ x if x is not None else 0 for x in data.data_s_65]
    # 'data_i_154' - ряд признаков, которые описывают устройство, с которого проводятся операции
    feat_matrix['data_i_154'] = [ x if x is not None else -150 for x in data.data_i_154]
    # 'cdf_s_124'- дата выдачи карты получателя
    feat_matrix['know_recip_card_age'] = ~data.cdf_s_124.isnull().astype(int)
    # в cdf_s_124 подмешана дата рождения, поэтому и max
    feat_matrix['recip_card_age'] = [max(x.days, 1000) if type(x) is not pd.tslib.NaTType else 1000 
                                     for x in (data.event_time - data.cdf_s_124)]

    feat_matrix['one_region'] = (data.cdf_s_20 == data.cdf_s_299).astype(int) # сравнение регионов

    # там с провайдером какая-то фигня до этого была (использовалась левая переменная)
    feat_matrix['ip_isp'] = data['ip_isp'].fillna(-1000000).astype(int)

    # оставшиеся признаки из исходного набора
    # сила связи отправителя и получателя
    feat_matrix['know_recip_power'] = [ x if x is not None else 0 for x in data.data_s_65] 
    feat_matrix['relative'] = [1 if x == u'ДА' else 0 for x in data.cdf_s_218] # перевод родственнику
    # разница возорастов получателей и отправителей, если отсутствует/неприменимо, то padding 500
    feat_matrix['age_diff'] = feat_matrix.client_age - [int(x) if x != 0 else 1000 for x in data.cdf_s_294] 

    #ADD NEW FEATURES
    # square of know_recip_power
    feat_matrix['krp_pow2'] = (feat_matrix['know_recip_power']) ** 2
    
    # natural log of amount
    feat_matrix['log_amount'] = np.log(feat_matrix['amount'] + 1)

    # len and code of region name
    feat_matrix['client_region_len'] = data.cdf_s_20.apply(lambda x: len(str(x)))
    feat_matrix['client_region'] = np.array([x if x.isdigit() else -1 for x in data.cdf_s_20], dtype=float)
    
    # 'cdf_s_136','cdf_s_137','cdf_s_140'- кумулятивные суммы операций за сутки в web, МП, web + МП 
    feat_matrix['amnt2chnls'] = (data["amount_original"].fillna(0) / \
        (data["cdf_s_136"].fillna(0).astype(float) + \
         data["cdf_s_136"].fillna(0).astype(float) + \
         data["cdf_s_140"].fillna(0).astype(float) + 1))

    # поставил order_cols временно, чтобы сохранить такой же порядок как в оригинальном
    order_cols = ['event_id', 'event_description', 'user_id', #'is_fraud',
                  'event_time', 'amount',
                  'client_age', 'cat_new_ip', 'cat_new_prov', 'channel_op', 'op_type', 
                  'age_diff', 'cumulative_sum_total', 'data_i_120',
                  'relative', 'know_recip_power', 'cdf_s_127', 'cdf_s_135', 'cdf_s_130',
                  'cdf_s_129', 'cdf_s_134', 'data_i_154', 'cdf_s_133',
                  'know_recip_card_age', 'recip_card_age', 'one_region', 'krp_pow2',
                  'log_amount', 'ip_isp', 'amnt2chnls']
    return feat_matrix[order_cols]

In [209]:
start_date = 20171029
end_date = 20171128

In [210]:
query = f"event_description in ({(str(include_channels))[1:-1]})" + \
        f" and short_date >= {str(start_date)}" + \
        f" and short_date <= {str(end_date)}"

In [211]:
query

"event_description in ('Перевод частному лицу', 'Оплата услуг', 'Перевод между своими счетами и картами') and short_date >= 20171029 and short_date <= 20171128"

In [300]:
def get_data(chunk_name):
    feat_test = features_handler(
        query = query,
        chunk_names=[chunk_name],
        calc_feat = calc_base_features)
    return feat_test

In [213]:
%%time
test_feats = get_data(train_files[5])

CPU times: user 38.1 s, sys: 2.31 s, total: 40.4 s
Wall time: 40.4 s


In [222]:
%%time
# create features matrix
with multiprocessing.Pool(processes=min(N_THREADS, FIRST_N)) as pool:
    results = pool.map(get_data, train_files)

CPU times: user 3.48 s, sys: 2.95 s, total: 6.43 s
Wall time: 9min 54s


In [225]:
total_df = pd.DataFrame()
for i, df in enumerate(results):
    total_df = pd.concat([total_df, df])
    #results[i] = 'bye'

In [226]:
total_df['short_date'] = total_df.event_time.apply(lambda x: x.date())
total_df.rename(columns={"is_fraud": "label"}, inplace=True)

In [227]:
total_df.short_date.min(), total_df.short_date.max()

(datetime.date(2017, 10, 29), datetime.date(2017, 11, 29))

In [228]:
total_df.groupby('label')['label'].count()

label
-1       3664
 0    3802643
 1      14012
Name: label, dtype: int64

In [230]:
total_df.groupby('event_description')['event_description'].count()

event_description
Оплата услуг                              1225172
Перевод между своими счетами и картами     736091
Перевод частному лицу                     1859056
Name: event_description, dtype: int64

In [231]:
itog = total_df.query("label != -1").copy()

In [232]:
n_uniques = np.apply_along_axis(lambda x: len(np.unique(x)), axis=0, arr=itog)

In [250]:
cat_feats = ["cat_new_ip", "cat_new_prov", "op_type", "relative",
             "cdf_s_127", "cdf_s_135", "cdf_s_130", "cdf_s_129", 
             "cdf_s_134", "cdf_s_133", "know_recip_card_age",
             "one_region"]
num_feats = ["amount", "client_age", "age_diff", "cumulative_sum_total",
             "cumulative_sum_total", "data_i_120", "know_recip_power",
             "data_i_120", "recip_card_age", "krp_pow2", "log_amount"]

# Create test dataset

In [293]:
test_files[:5]

['../../data/raw_splits/test/chunk_0.fth',
 '../../data/raw_splits/test/chunk_1.fth',
 '../../data/raw_splits/test/chunk_2.fth',
 '../../data/raw_splits/test/chunk_3.fth',
 '../../data/raw_splits/test/chunk_4.fth']

In [305]:
test_start_date = 20171129
test_end_date =  20171205

In [306]:
query = f"event_description in ({(str(include_channels))[1:-1]})" + \
        f" and short_date >= {str(start_date)}" + \
        f" and short_date <= {str(end_date)}"

In [307]:
def get_data(chunk_name):
    feat_test = features_handler(
        query = query,
        chunk_names=[chunk_name],
        calc_feat = calc_base_features)
    return feat_test

In [309]:
%%time
# create features matrix
with multiprocessing.Pool(processes=min(3, FIRST_N)) as pool:
    res_test = pool.map(get_data, test_files)

CPU times: user 1.77 s, sys: 1.44 s, total: 3.21 s
Wall time: 2min 12s


In [310]:
total_df_test = pd.DataFrame()
for i, df in enumerate(res_test):
    total_df_test = pd.concat([total_df_test, df])
    #results[i] = 'bye'

In [315]:
itog_test = total_df_test.copy()
itog_test.drop('channel_op', inplace=True)

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [316]:
itog.shape

(3816655, 30)

In [317]:
itog_test.shape

(2035724, 29)

In [261]:
itog[num_feats].dtypes

amount                  float64
client_age              float64
age_diff                float64
cumulative_sum_total    float64
cumulative_sum_total    float64
data_i_120                int64
know_recip_power          int64
data_i_120                int64
recip_card_age            int64
krp_pow2                  int64
log_amount              float64
dtype: object

In [260]:
itog[cat_feats].dtypes

cat_new_ip             int64
cat_new_prov           int64
op_type                int64
relative               int64
cdf_s_127              int64
cdf_s_135              int64
cdf_s_130              int64
cdf_s_129              int64
cdf_s_134              int64
cdf_s_133              int64
know_recip_card_age    int64
one_region             int64
dtype: object

In [291]:
# channel_op has only one unique values
pd.DataFrame({"feature": itog.columns.tolist(), "n_uniq": (n_uniques.tolist())})
itog.drop('channel_op', axis=1, inplace=True)

In [318]:
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

class counter_cv:

    def __init__(self,
                 alpha,
                 cv_internal=3,
                 cv_external=3):

        self._cv_internal = cv_internal
        self._cv_external = cv_external
        self._alpha = alpha

    def _map_rule_cv(self, x, y, unique_x):
        """
        create cv map rule
        """

        map_rule_res = None

        counter = pd.DataFrame({'x': x, 'y': y})
        kf = KFold(n_splits=self._cv_internal)

        for train_ind, test_ind in kf.split(x):
            # mean for regularization
            train_mean = np.mean(y[train_ind])

            mean_by_cat = counter.iloc[train_ind, :].groupby('x')['y'].mean()
            count_by_cat = counter.iloc[train_ind, :].groupby('x')['y'].count()

            # smooth mean for categories
            map_rule = ((mean_by_cat * count_by_cat +
                         self._alpha * train_mean) /
                        (count_by_cat + self._alpha))

            # add missed categories to map rule
            missed_cat = list(set(unique_x) - set(map_rule.index))
            num_missed_cat = len(missed_cat)

            if (num_missed_cat != 0):
                map_rule.append(
                    pd.Series(np.repeat(train_mean, num_missed_cat),
                              index=missed_cat))
            else:
                pass

            # create or update resulted map_rule
            if (map_rule_res is None):
                map_rule_res = map_rule / self._cv_internal
            else:
                map_rule_res += map_rule / self._cv_internal

        return(map_rule_res)

    def map_test(self, x_train, y_train, x_test):
        """
        map categorial feature (x_test) based on mean values
        of y in train sample

        return numpy array, shape like x_test
        """

        x_train = x_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        x_test = x_test.reset_index(drop=True)

        # make pandas data frame for grouping and create map rule
        counter = pd.DataFrame({'x': x_train, 'y': y_train})
        map_rule = counter.groupby('x')['y'].mean()

        return(x_test.map(map_rule))

    def map_train(self, x, y):
        """
        map categorial feature (x_train) based on double cross-validated
        mean values

        return numpy array, shape like y_train
        """

        unique_x = np.unique(x)

        x = x.reset_index(drop=True)
        y = y.reset_index(drop=True)

        kf = KFold(n_splits=self._cv_external)

        blank = np.zeros_like(y, dtype='float32')
        for train_ind, test_ind in kf.split(x):
            map_rule = self._map_rule_cv(x=x[train_ind],
                                         y=y[train_ind],
                                         unique_x=unique_x)

            blank[test_ind] = x[test_ind].map(map_rule)

        return(blank)

In [323]:
itog.to_csv("../../data/coms_sep/train_no_cnts.csv", index=False)
itog_test.to_csv("../../data/coms_sep/test_no_cnts.csv", index=False)

In [324]:
for i in cat_feats:
    tt = TicToc()
    tt.tic()
    counter = counter_cv(alpha=1000)
    itog[i] = counter.map_train(itog[i], itog['label'])
    itog_test[i] = counter.map_test(itog[i], itog['label'], itog_test[i])
    tt.toc()

/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Elapsed time is 10.025378 seconds.
Elapsed time is 10.085192 seconds.
Elapsed time is 9.586066 seconds.
Elapsed time is 10.024778 seconds.
Elapsed time is 9.120979 seconds.
Elapsed time is 9.546427 seconds.
Elapsed time is 9.326977 seconds.
Elapsed time is 9.585274 seconds.
Elapsed time is 10.078037 seconds.
Elapsed time is 10.970302 seconds.
Elapsed time is 9.275317 seconds.
Elapsed time is 9.824616 seconds.


In [327]:
itog.to_csv("../../data/coms_sep/train.csv", index=False)
itog_test.to_csv("../../data/coms_sep/test.csv", index=False)

In [ ]:
itog.to_csv("../data/coms_sep/train.csv")
# total_df = pd.read_csv("../data/coms_sep/train.csv", index_col=0)

In [273]:
x = itog[['know_recip_card_age']]

In [275]:
x.groupby('know_recip_card_age')['label'].mean()

KeyError: 'Column not found: label'

In [265]:
isinstance(itog['user_id'], pd.Series)

True

In [285]:
counter = counter_cv(alpha=1000)

In [287]:
keka = counter.map_train(x=itog['op_type'], y=itog['label'])

/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [269]:
class CouterEncoder:
    def __init__(alpha, feats, target):
        self._alpha = alpha
    
    def fit(self, X, y):
        if not isinstance(x, pd.DataFrame):
            return "X should be data frame"
        self._global_mean = np.mean(x)
        self._counters = x.groupby()[target_name].mean()
        self._n_counters = data.groupby()[target_name].count()
        return self

    def transform(self, x):
        if not isinstance(x, pd.DataFrame):
            return "X should be data frame"
            x = pd.Series(x)
        out = ((x.map(self._counters) + self._global_mean * alpha) / \
               (x.map(self._n_counters) + alpha))
        return out
    
    def fit_transform(x, y):
        self.fit(x, y)
        return self.transform(x)

In [ ]:
def map_counter(data, field_name, target_name, alpha, global_mean=None):
    if global_mean is None:
        global_mean = data[target_name].mean()
    counters = data.groupby(field_name)[target_name].mean()
    n_counters = data.groupby(field_name)[target_name].count()
    out = ((data[field_name].map(counters) + global_mean * alpha)/ \
           (data[field_name].map(n_counters) + alpha))
    return out

In [ ]:
n_uniques

In [ ]:
total_df.columns

In [ ]:
for col in total_df.columns:
    col_name = total_df.columns[i]
    if col_name not in ['event_id', "user_id", "event_time", "label", "short_date"]:
        total_df[col] = map_counter(total_df, col, 'label',
                                    alpha=1000,
                                    global_mean=global_mean)

In [ ]:
#total_df.to_csv("../data/coms_sep/train_cntrs.csv")
#total_df = pd.read_csv("../data/coms_sep/train_cntrs.csv", index_col=0)

In [ ]:
total_df.label.value_counts()